In [16]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [17]:
from tra.models import *

In [18]:
queryset = TestRun.objects.all()
regfilters = TestSetFilter.objects.all()
queryset = queryset.filter(
    reduce(lambda q, reg_filter: q | Q(testline_type=reg_filter.testline_type, 
                                       test_instance__test_set=reg_filter), regfilters, Q())
)

In [19]:

def get_distinct_values_and_serialize(field, model, serializer=None, order_by_param=None, key_override=None):
    order_by_param = order_by_param if order_by_param else field
    distinct_values = queryset.order_by(order_by_param).distinct(field).values_list(field, flat=True)
    objects = model.objects.filter(pk__in=distinct_values)
    data = serialize("json", objects)
    key = field if not key_override else key_override
    return json.loads(data)

In [32]:
# get_distinct_values_and_serialize('test_instance__test_set__test_set_name', TestSetFilter, key_override="test_set_name",
#                                    order_by_param="test_instance__test_set__test_set_name")

distinct = queryset.order_by('test_instance__test_set__test_set_name').distinct('test_instance__test_set__test_set_name').values_list('test_instance__test_set__test_set_name', flat=True)
print(distinct)
# TestSetFilter.objects.filter(pk__in=distinct)
serialize("json", distinct)

<QuerySet ['5GC001085-A_Intra-frequency_intra-gNB_neighbor_NRREL_addition', '5GC001085-B_Intra-frequency_inter-gNB_neighbor_NRREL_addition_-_Previously_established_Xn', '5GC001085-C_Intra-frequency_NRREL_deletion_(Pruning)_based_on_Measurement_Reports', '5GC002175-QB', '5GC002175-RB', '5GC002640_AD_DRX_MG_support_IODT', 'CB007883-5G-D_-_Spillover_of_5GC001085-D_Intra_Frequency_Inter-gNB_neighbor_NRREL_and_Xn_addition_SON_Config_Transfer', 'CB008533_B_Multiple_UEs_Call_Setup_4CC_DL_CA']>


AttributeError: 'str' object has no attribute '_meta'

In [38]:
distinct = regfilters.order_by('test_set_name').distinct('test_set_name').values_list('test_set_name', flat=True)

In [40]:
[{'pk': elem} for elem in list(distinct)]

[{'pk': '5GC001085-A_Intra-frequency_intra-gNB_neighbor_NRREL_addition'},
 {'pk': '5GC001085-B_Intra-frequency_inter-gNB_neighbor_NRREL_addition_-_Previously_established_Xn'},
 {'pk': '5GC001085-C_Intra-frequency_NRREL_deletion_(Pruning)_based_on_Measurement_Reports'},
 {'pk': '5GC002175-QB'},
 {'pk': '5GC002175-RB'},
 {'pk': '5GC002640_AD_DRX_MG_support_IODT'},
 {'pk': 'CB007883-5G-D_-_Spillover_of_5GC001085-D_Intra_Frequency_Inter-gNB_neighbor_NRREL_and_Xn_addition_SON_Config_Transfer'},
 {'pk': 'CB008533_B_Multiple_UEs_Call_Setup_4CC_DL_CA'}]